In [1]:
import xarray as xr
import numpy as np
import functions as fun
import scipy.stats as st
import matplotlib.pyplot as plt

In [2]:
#mods = fun.models()
mods = ['bcc-csm1-1','CanESM2','CCSM4','CNRM-CM5','CSIRO-Mk3-6-0','GFDL-CM3','GISS-E2-R','HadGEM2-ES','inmcm4','IPSL-CM5A-LR','MIROC-ESM','MIROC5','MRI-CGCM3','NorESM1-M']

In [3]:
#Get 1x1 degree grid
with xr.open_dataset('../rawdata/rsl_85_2090.nc') as ds:
    lon = ds['x'].values
    lat = ds['y'].values
    mask = ds['slr_md'].values

#Get fingerprints
with xr.open_dataset(f'../data/fingerprints.nc') as ds:
    fp_a = ds['fp_a'].values
    fp_ea = ds['fp_ea'].values
    fp_wa = ds['fp_wa'].values
    fp_gr = ds['fp_gr'].values
    fp_gl = ds['fp_gl'].values

In [4]:
#Response functions Antarctic dynamics
basin = ['eais','wedd','amun','ross','apen']
imods = ['AISM_VUB','BISI_LBL','CISM_NCA','FETI_VUB','GRIS_LSC','IMAU_VUB','ISSM_JPL','ISSM_UCI','MALI_LAN','PISM_AWI','PISM_DMI','PISM_PIK','PISM_VUW','PS3D_PSU','SICO_UHO','UA_UNN']

R = np.zeros((len(imods),len(basin),200))
for b,bb in enumerate(range(1,6)):
    for i,imod in enumerate(imods):
        with open(f'../rawdata/Larmip2019-master/RFunctions/RF_{imod}_BM08_R{bb}.dat') as f:
            R[i,b,:] = np.array([float(x) for x in f.readlines()])

In [5]:
#Sterodynamic
def estdn(scen,N):
    ens = np.zeros((len(mods),len(lat),len(lon)))
    gens = np.zeros((len(mods),N))

    for m,model in enumerate(mods):
        #Get steric expansion    
        with xr.open_dataset(f'../data/steric/{model}_{scen}.nc') as ds:
            slr = ds['SLR'].values
            gens[m,:] = np.nanmean(slr[-20:])
    
        #Get zos
        with xr.open_dataset(f'../rawdata/zos/CorrectedZOS_EXP{scen}_{model}.nc') as ds:
            slr = ds['CorrectedZOS_reg'].values
            ens[m,:,:] = np.nanmean(slr[-20:,:,:],axis=0) + gens[m,0]
            ens[m,:,:][np.isnan(mask)] = np.nan
    
    print(scen,'got stdn')
    return np.repeat(ens[:,np.newaxis,:,:],N,axis=1),gens

In [6]:
#Glaciers
def eglac(scen,N):
    ens = np.zeros((len(mods),len(lat),len(lon)))
    gens = np.zeros((len(mods),N))

    for m,model in enumerate(mods):
        #Get global
        with xr.open_dataset(f'../data/glacier/{model}_{scen}.nc') as ds:
            slr = ds['SLR'].values
            gens[m,:] = np.nanmean(slr[-20:])
        #Get regional
        for r in range(1,20):
            with xr.open_dataset(f'../data/glacier/{model}_{scen}_{r}.nc') as ds:
                slr = np.nanmean(ds['SLR'][-20:].values,axis=0)
                ens[m,:,:] += slr*fp_gl[r-1,:,:]    

    #Add process variance to global
    std = np.nanstd(gens[:,0])
    gens += np.random.normal(loc=0,scale=std,size=(len(mods),N))

    print(scen,'got glac')
    return np.repeat(ens[:,np.newaxis,:,:],N,axis=1),gens

In [7]:
#Greenland SMB
def egsmb(scen,N):
    ens = np.zeros((len(mods),N,len(lat),len(lon)))
    gens = np.zeros((len(mods),N))

    for m,model in enumerate(mods):

        with xr.open_dataset(f'../rawdata/Gsmb_vars/{model}_{scen}.nc') as ds:
            TA = ds['TA'].values
            SF = ds['SF'].values
            years = ds['time'].values
        TA = TA[years<2101]
        SF = SF[years<2101]
        years = years[years<2101]            
            
        MW  = 84.2*TA + 2.4*TA**2. + 1.6*TA**3.
        SMB = SF-MW

        #Skewed uncertainty following AR5
        F = np.exp(np.random.normal(loc=0.0, scale=0.4, size=N))
        E = np.random.uniform(1,1.15,size=N)
        for n in range(N):
            SLR = -np.cumsum(E[n]*F[n]*SMB)*2.8e-4 #in cm
            SLR = fun.detrend(SLR,np.array([]),years) #Reference to ref period
            gens[m,n] = np.nanmean(SLR[-20:])
            ens[m,n,:,:] = np.nanmean(SLR[-20:])*fp_gr

    print(scen,'got gsmb')
    return ens,gens

In [8]:
#Antarctic SMB
def easmb(scen,N):
    ens = np.zeros((len(mods),N,len(lat),len(lon)))
    gens = np.zeros((len(mods),N))
    
    for m,model in enumerate(mods):

        with xr.open_dataset(f'../rawdata/Asmb_vars/{model}_{scen}.nc') as ds:
            TA = ds['TA'].values
            years = ds['time'].values
            
        TA = TA[years<2101]
        years = years[years<2101]
        #Uncertainty following AR5
        P = np.random.normal(loc=0.051, scale = 0.015,size=N)
        S = np.random.uniform(0,0.035,size=N)
        for n in range(N):
            SMB = 1923*P[n]*(1-S[n])*TA
            SMB[years<1995]=0
            SLR = -np.cumsum(SMB)*2.8e-4 #in cm
            gens[m,n] = np.nanmean(SLR[-20:])
            ens[m,n,:,:] = np.nanmean(SLR[-20:])*fp_a

    print(scen,'got asmb')
    return ens,gens

In [9]:
#Antarctic dynamics
def eadyn(scen,N):
    ens = np.zeros((len(mods),N,len(lat),len(lon)))
    gens = np.zeros((len(mods),N))
    
    for m,model in enumerate(mods):  

        with xr.open_dataset(f'../rawdata/Adyn_temp/{model}_{scen}.nc') as ds:
            TH = ds['TH'].values
            years = ds['time'].values

        #Uncertainty following Levermann et al 2020
        bms = np.random.uniform(7,16,N)
        ii  = np.random.randint(len(imods), size=N)

        for n in range(N):   
            for b,bb in enumerate(range(1,6)):
                SLR = np.zeros(len(years))
                for t,tt in enumerate(years):
                    if t==0: continue
                    SLR[t] = bms[n]*100*np.sum(TH[b,:t][::-1]*R[ii[n],b,:t])
                SLR = fun.detrend(SLR,np.array([]),years)
                gens[m,n] += np.nanmean(SLR[-20:])
                if b in [0,1,3]:
                    ens[m,n,:,:] += np.nanmean(SLR[-20:])*fp_ea
                else:
                    ens[m,n,:,:] += np.nanmean(SLR[-20:])*fp_wa
            
    print(scen,'got adyn')
    return ens,gens

In [10]:
N = 100
it = np.arange(N)
for s,scen in enumerate(['rcp45','rcp85']):

    #Save regional ensemble
    eestdn,gestdn = estdn(scen,N)
    eeglac,geglac = eglac(scen,N)
    eegsmb,gegsmb = egsmb(scen,N)
    eeasmb,geasmb = easmb(scen,N)
    eeadyn,geadyn = eadyn(scen,N)
    
    eestdn2 = xr.DataArray(eestdn,dims=('model','iter','lat','lon'),coords={'model':mods,'iter':it,'lat':lat,'lon':lon})
    eeglac2 = xr.DataArray(eeglac,dims=('model','iter','lat','lon'),coords={'model':mods,'iter':it,'lat':lat,'lon':lon})
    eegsmb2 = xr.DataArray(eegsmb,dims=('model','iter','lat','lon'),coords={'model':mods,'iter':it,'lat':lat,'lon':lon})   
    eeasmb2 = xr.DataArray(eeasmb,dims=('model','iter','lat','lon'),coords={'model':mods,'iter':it,'lat':lat,'lon':lon})
    eeadyn2 = xr.DataArray(eeadyn,dims=('model','iter','lat','lon'),coords={'model':mods,'iter':it,'lat':lat,'lon':lon})
    
    ds = xr.Dataset({'stdn':eestdn2,'glac':eeglac2,'gsmb':eegsmb2,'asmb':eeasmb2,'adyn':eeadyn2})
    ds.to_netcdf(f'../data/ensemble_reg_{scen}.nc')
    
    #Save global ensemble
    gestdn2 = xr.DataArray(gestdn,dims=('model','iter'),coords={'model':mods,'iter':it})
    geglac2 = xr.DataArray(geglac,dims=('model','iter'),coords={'model':mods,'iter':it})
    gegsmb2 = xr.DataArray(gegsmb,dims=('model','iter'),coords={'model':mods,'iter':it}) 
    geasmb2 = xr.DataArray(geasmb,dims=('model','iter'),coords={'model':mods,'iter':it})
    geadyn2 = xr.DataArray(geadyn,dims=('model','iter'),coords={'model':mods,'iter':it})
    
    ds = xr.Dataset({'stdn':gestdn2,'glac':geglac2,'gsmb':gegsmb2,'asmb':geasmb2,'adyn':geadyn2})
    ds.to_netcdf(f'../data/ensemble_glob_{scen}.nc')
    
    print('--------------------------------')
    
    #Table 2 values
    mstdn = np.zeros(len(mods))
    mglac = np.zeros(len(mods))
    mgsmb = np.zeros(len(mods))
    masmb = np.zeros(len(mods))
    madyn = np.zeros(len(mods))
    for m,model in enumerate(mods):
        mstdn[m] = np.median(gestdn[m,:])
        mglac[m] = np.median(geglac[m,:])
        mgsmb[m] = np.median(gegsmb[m,:])
        masmb[m] = np.median(geasmb[m,:])
        madyn[m] = np.median(geadyn[m,:])
        
        print(f"{model:10} & {mstdn[m]:.0f} & {mglac[m]:.0f} & {mgsmb[m]:.0f} & {masmb[m]:.0f} & {madyn[m]:.0f} & {(mstdn[m]+mglac[m]+mgsmb[m]+masmb[m]+madyn[m]):.0f} \ ")
    print(f"Multi-model mean & {np.mean(mstdn):.0f} & {np.mean(mglac):.0f} & {np.mean(mgsmb):.0f} & {np.mean(masmb):.0f} & {np.mean(madyn):.0f} & {np.mean(mstdn+mglac+mgsmb+masmb+madyn):.0f} \ ")
    print(f'')
    print(f"Variance (cm$^2$) & {np.var(mstdn):.0f} & {np.var(mglac):.0f} & {np.var(mgsmb):.0f} & {np.var(masmb):.0f} & {np.var(madyn):.0f} & {np.var(mstdn+mglac+mgsmb+masmb+madyn):.0f}")
    print(f'')
    print('---------------------------------')
    

/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: Mean of empty slice
  from ipykernel import kernelapp as app


rcp45 got stdn
rcp45 got glac
rcp45 got gsmb
rcp45 got asmb
rcp45 got adyn
--------------------------------
bcc-csm1-1 & 15 & 15 & 1 & -3 & 3 & 31 \ 
CanESM2    & 20 & 18 & 4 & -4 & 5 & 42 \ 
CCSM4      & 18 & 16 & -1 & -3 & 11 & 40 \ 
CNRM-CM5   & 13 & 16 & 2 & -3 & 11 & 38 \ 
CSIRO-Mk3-6-0 & 20 & 12 & 3 & -4 & 4 & 34 \ 
GFDL-CM3   & 26 & 22 & 9 & -2 & 6 & 60 \ 
GISS-E2-R  & 22 & 12 & 2 & -2 & 6 & 40 \ 
HadGEM2-ES & 17 & 21 & 5 & -4 & 5 & 43 \ 
inmcm4     & 15 & 7 & 1 & -3 & 8 & 29 \ 
IPSL-CM5A-LR & 17 & 18 & 5 & -2 & 6 & 43 \ 
MIROC-ESM  & 23 & 18 & 7 & -3 & 3 & 48 \ 
MIROC5     & 19 & 16 & 5 & -2 & 6 & 44 \ 
MRI-CGCM3  & 13 & 10 & 0 & -3 & 6 & 27 \ 
NorESM1-M  & 19 & 17 & 2 & -2 & 8 & 44 \ 
Multi-model mean & 18 & 15 & 3 & -3 & 6 & 40 \ 

Variance (cm$^2$) & 12 & 16 & 7 & 1 & 6 & 67

---------------------------------
rcp85 got stdn
rcp85 got glac
rcp85 got gsmb
rcp85 got asmb
rcp85 got adyn
--------------------------------
bcc-csm1-1 & 24 & 20 & 3 & -4 & 3 & 45 \ 
CanESM2    & 29 & 